In [1]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import pandas as pd
import sys
import StringIO as sio
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import base

In [2]:
import ujson as json
import gzip
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    data = [json.loads(line) for line in f]

In [3]:
print len(data)
print type(data)
print data[0]

37938
<type 'list'>
{u'city': u'Phoenix', u'review_count': 7, u'name': u'Eric Goldberg, MD', u'neighborhoods': [], u'type': u'business', u'business_id': u'vcNAWiLM4dR7D2nwwJ7nCA', u'full_address': u'4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018', u'hours': {u'Thursday': {u'close': u'17:00', u'open': u'08:00'}, u'Tuesday': {u'close': u'17:00', u'open': u'08:00'}, u'Friday': {u'close': u'17:00', u'open': u'08:00'}, u'Wednesday': {u'close': u'17:00', u'open': u'08:00'}, u'Monday': {u'close': u'17:00', u'open': u'08:00'}}, u'state': u'AZ', u'longitude': -111.983758, u'stars': 3.5, u'latitude': 33.499313, u'attributes': {u'By Appointment Only': True}, u'open': True, u'categories': [u'Doctors', u'Health & Medical']}


In [4]:
star_ratings = [row['stars'] for row in data]
city = [row['city'] for row in data]
nbrhood = [row['neighborhoods'] for row in data]
lat = [row['latitude'] for row in data]
lon = [row['longitude'] for row in data] 
cat= [row['categories'] for row in data]
print len(star_ratings),len(city),len(nbrhood),len(lat),len(lon),len(cat)

37938 37938 37938 37938 37938 37938


In [5]:
X_tr=[]
y_tr=[]
for i in range(37938):
    X_tr.append((city[i],nbrhood[i],lat[i],lon[i],cat[i]))
    y_tr.append(star_ratings[i])

In [12]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        out=[]
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        for row in X:
            out.append([row[i] for i in self.col_names])
        return out

In [13]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        ret_list=[]
        for i in range(len(X)):
            dic={}
            for j in range(len(X[i][0])):
                dic[X[i][0][j]]=1
            ret_list.append(dic)
        return ret_list

In [14]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
cst=ColumnSelectTransformer(['categories','city','latitude','longitude'])
d_en=DictEncoder()
dv = DictVectorizer(sparse=False)
tfid=TfidfTransformer()
sgdr=SGDRegressor(verbose=2,max_iter=1)
category_est = Pipeline([
        ('cst',ColumnSelectTransformer(['categories','city','latitude','longitude'])),
        ('d_en',DictEncoder()),
        ('dv',DictVectorizer(sparse=False)),
        ('tfid',TfidfTransformer()),
        ('sgdr',SGDRegressor(verbose=2,max_iter=1))
    ])   

In [16]:
category_est.fit(data, star_ratings)

-- Epoch 1
Norm: 10.87, NNZs: 706, Bias: 0.730671, T: 37938, Avg. loss: 2.566774
Total training time: 0.00 seconds.


Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(col_names=['categories', 'city', 'latitude', 'longitude'])), ('d_en', DictEncoder()), ('dv', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('tfid', TfidfTransformer(norm=u'l2', smooth_idf=True, sublinear_tf=False...      power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=2,
       warm_start=False))])